In [1]:
import joblib
import numpy as np
import pandas as pd

from pymatgen import Structure

# user-friendly print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Crystal structure generation

To generate legal structure under a given space group with specific chemical composition, basicly the following four steps are needed.

1. calculate possible Wyckoff configurations under a given space group for each chemical composition.
2. generate fraction positions for each element with given a Wyckoff configuration which is calculated from step 1), randomly.
3. generate lattice for the given space group which is used in step 1), randomly.
4. combine the results from step 2) and 3) to obtain a legal structure.

Usually, we also have to check the `volume` and `atomic distances` of generated structure, only keep the structures which have reasonable `volume` and `atomic distances`.

To facilitate these jobs, our `crystallus` library provides four modules:

* `WyckoffCfgGenerator`: generate possible Wyckoff configurations for the given space group and composition of primitive cell.
* `CrystalGenerator`: generate crystal structures for the given space group and Wyckoff configurations.
* `WyckoffDB, SpaceGroupDB`: database include space group and corresponding Wyckoff information.

We will show how to use `crystallus` to generate legal structures.

### 1. generate Wyckoff configurations

As an example, we will try to generate structures for `Ca2C2O6`. The true space group of this structure is `167`, and the Wyckoff configuration is `{Ca: 2b, C: 2a, O: 6e}`.
You can use `SpaceGroupDB` to get the Wyckoff information about space group `167`.

In [12]:
from crystallus import SpaceGroupDB

wys = SpaceGroupDB.get(spacegroup_num=167).wyckoffs
[{'Wyckoff letter': w.letter, 'multiplicity': w.multiplicity, 'reusable': w.reuse, 'Wyckoff position': w.positions} for w in wys ]

[{'Wyckoff letter': 'f',
  'multiplicity': 12,
  'reusable': True,
  'Wyckoff position': '(x,y,z), (z,x,y), (y,z,x), (-y+1/2,-x+1/2,-z+1/2), (-x+1/2,-z+1/2,-y+1/2), (-z+1/2,-y+1/2,-x+1/2), (-x,-y,-z), (-z,-x,-y), (-y,-z,-x), (y+1/2,x+1/2,z+1/2), (x+1/2,z+1/2,y+1/2), (z+1/2,y+1/2,x+1/2)'},
 {'Wyckoff letter': 'e',
  'multiplicity': 6,
  'reusable': True,
  'Wyckoff position': '(x,-x+1/2,1/4), (1/4,x,-x+1/2), (-x+1/2,1/4,x), (-x,x+1/2,3/4), (3/4,-x,x+1/2), (x+1/2,3/4,-x)'},
 {'Wyckoff letter': 'd',
  'multiplicity': 6,
  'reusable': False,
  'Wyckoff position': '(1/2,0,0), (0,1/2,0), (0,0,1/2), (1/2,0,1/2), (0,1/2,1/2), (1/2,1/2,0)'},
 {'Wyckoff letter': 'c',
  'multiplicity': 4,
  'reusable': True,
  'Wyckoff position': '(x,x,x), (-x+1/2,-x+1/2,-x+1/2), (-x,-x,-x), (x+1/2,x+1/2,x+1/2)'},
 {'Wyckoff letter': 'b',
  'multiplicity': 2,
  'reusable': False,
  'Wyckoff position': '(0,0,0), (1/2,1/2,1/2)'},
 {'Wyckoff letter': 'a',
  'multiplicity': 2,
  'reusable': False,
  'Wyckoff position

Let's generate some possible Wyckoff configurations for compositon `Ca2C2O6` under space group `167`.

In [16]:
from crystallus import WyckoffCfgGenerator

WyckoffCfgGenerator?

In [19]:
composition = {'Ca': 2, 'C': 2, 'O': 6}

wyg = WyckoffCfgGenerator(**composition)
wyg

WyckoffCfgGenerator(            
    max_recurrent=1000,            
    n_jobs=-1            
    composition={'Ca': 2, 'C': 2, 'O': 6}            
)

You can see that the minimum input to initialize a `WyckoffCfgGenerator` is just the compositon.

In [29]:
cfg = wyg.gen_one(spacegroup_num=167)
cfg

{'C': ['b'], 'Ca': ['a'], 'O': ['e']}

In [ ]:
from crystallus import CrystalGenerator, WyckoffCfgGenerator

In [ ]:
composition = 'Ti4O8'
estimated_volume = 146.706
estimated_variance = 20
sp_num = 12

In [ ]:
cg = CrystalGenerator(sp_num, estimated_volume, estimated_variance)
cg

In [ ]:
len(cfgs[sp_num])

In [ ]:
%%time

ret = cg.gen_many(10, *cfgs[sp_num])

In [ ]:
%%time

len(ret)
joblib.dump(ret, f"{composition}_space_group_{sp_num}.pkl.z")

In [ ]:
for i, tmp in enumerate(ret):
    s = Structure(lattice=tmp['lattice'], species=tmp['species'], coords=np.asarray(tmp['coords']).reshape(-1, 3))
    s.to(fmt='cif', filename=f'generated_cifs/{i}.cif', symprec=0.01)

---------

In [ ]:
structure_table = pd.read_pickle('Ti4O8_structure_table_old.pd.xz')
structure_table

In [ ]:
tmp = structure_table.groupby('spacegroup_num')['wy_letters'].value_counts()

In [ ]:
from matplotlib import pyplot as plt

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8),dpi=100)
structure_table.shape
ax = structure_table.groupby('spacegroup_num').count().sort_values('formula', ascending=False).plot.bar(y=['structure'], ax=ax1)
ax.text(25,190000,'Ca2C2O6\nsize: 624899', fontsize=15, ha='right')

ax = structure_table.volume.hist(ax=ax2)
ax.xaxis.grid(False)
ax.grid(linestyle='--', linewidth=1, axis='y')
ax.set_xlabel('volume')

plt.tight_layout()